In [ ]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_max_distance_to_target_surface, contact_optimization, show_selected_joints, highlight_rod_and_joint)

import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
scale = 1
rod_length = 0.07244622391567243 * scale
width = rod_length / 20 * 10 
thickness = width / 10 * 0.35 
print(width, thickness)
io = InputOrganizer('kleinbottle_projected_{}'.format(scale), thickness, width, weaving_dir)

In [ ]:
rgb_scale = 255
cmyk_scale = 100
def cmyk_to_rgb(c,m,y,k):
    """
    """
    r = (1.0-(c+k)/float(cmyk_scale))
    g = (1.0-(m+k)/float(cmyk_scale))
    b = (1.0-(y+k)/float(cmyk_scale))
    return r,g,b

def get_blue_pink_color(colors):
    cmyk_colors = colors * 100
    colors = [list(cmyk_to_rgb(100 - cmyk_colors[i], 33, 0, 0)) + [0] for i in range(len(cmyk_colors))]
    return colors

def get_color_scheme(colors):
    cmap = plt.cm.plasma
#     cmap = cmap2
    return cmap(colors)

In [ ]:
cmyk_to_rgb(100, 33, 0, 0)

In [ ]:
with open('./plasma_color_bar.obj', 'w') as f:
    res = 2000
    colors = get_color_scheme(np.linspace(0, 1, res))
    for i in range(res):
        f.write('v {} {} {} {} {} {}\n'.format(i/(res / 100.), 0, 0, colors[i][0], colors[i][1], colors[i][2]))
        f.write('v {} {} {} {} {} {}\n'.format(i/(res / 100.), 100. / 10, 0, colors[i][0], colors[i][1], colors[i][2]))
    for i in range(res - 1):
        f.write('f {} {} {}\n'.format(i * 2 + 1, i * 2 + 2, i * 2 + 3))
        f.write('f {} {} {}\n'.format(i * 2 + 2, i * 2 + 3, i * 2 + 4))

In [ ]:
with open('/Users/yren/Develop/EPFL_LGG/weaving/weaving/weaving_editor/examples/kleinbottle_nature_figure3/kleinbottle_nature_figure3_straight_linkage.obj') as l:
    content = l.readlines()
    colors = []
    for line in content:
        if 'vd ' in line:
            colors.append(float(line.split(' ')[1]))
colors = np.array(colors)
colors = colors / 0.66
print(max(colors))
colors = get_color_scheme(colors)
with open('kleinbottle_straight_linkage_with_color.obj', 'w') as f:
    vx_count = 0
    for line in content:
        if 'v ' in line:
            point = np.array([float(x) for x in line.split(' ')[1:]])
            f.write('v {} {} {} {} {} {}\n'.format(point[0], point[1], point[2], colors[vx_count][0], colors[vx_count][1], colors[vx_count][2]))
            vx_count += 1
        else:
            f.write(line)
        

In [ ]:
with open('/Users/yren/Develop/EPFL_LGG/weaving/weaving/weaving_editor/examples/kleinbottle_nature_figure3/kleinbottle_nature_figure3_curved_linkage.obj') as l:
    content = l.readlines()
    colors = []
    for line in content:
        if 'vd ' in line:
            colors.append(float(line.split(' ')[1]))
colors = np.array(colors)
colors = colors / 0.66
colors = get_color_scheme(colors)
with open('kleinbottle_curved_linkage_with_color.obj', 'w') as f:
    vx_count = 0
    for line in content:
        if 'v ' in line:
            point = np.array([float(x) for x in line.split(' ')[1:]])
            f.write('v {} {} {} {} {} {}\n'.format(point[0], point[1], point[2], colors[vx_count][0], colors[vx_count][1], colors[vx_count][2]))
            vx_count += 1
        else:
            f.write(line)
        

In [ ]:
246/255, 144/255, 109/255

In [ ]:
255/255,16/255,83/255

In [ ]:
cmap1 = plot.Colormap('violet red', name='violet', fade=100, space='hsl')

In [ ]:
cmap1(256)

In [ ]:
import colorsys

In [ ]:
cmap1.N

In [ ]:
cmap1(256)

In [ ]:
cmap1(0)

In [ ]:
color = cmap1(256)

In [ ]:
colorsys.hls_to_rgb(color[0], color[2], color[1])

In [ ]:
plot.to_rgba(cmap1(1), space = 'hsl')

In [ ]:
fig, axs = plot.show_cmaps(plt.cm.PuRd)

In [ ]:
fig.savefig('PuRd.svg')

In [ ]:
print(np.max(data))

In [ ]:
import proplot as plot
import numpy as np
state = np.random.RandomState(51423)
data = state.rand(30, 30).cumsum(axis=1)
# Initialize figure
fig, axs = plot.subplots([[1, 1, 2, 2], [0, 3, 3, 0]], axwidth=2, span=0)
axs.format(
    xlabel='x axis', ylabel='y axis',
    suptitle='Building your own PerceptuallyUniformColormaps'
)

# Colormap from named color
# The trailing '_r' makes the colormap go dark-to-light instead of light-to-dark
cmap1 = plot.Colormap('violet red', name='pacific', fade=100, space='hsl')
axs[0].format(title='From single named color')
axs[0].pcolormesh(data, cmap=cmap1)

# Colormap from lists
cmap2 = plot.Colormap(((338/360., 91/100., 65/100.), (20/360., 86/100., 92/100.)), name='heatwave', listmode = 'perceptual', space = 'hsl')
axs[1].format(title='From list of colors')
axs[1].pcolormesh(data, cmap=cmap2)

# Colormaps from channel value dictionaries
cmap3 = plot.Colormap(
    {
        'hue': ['red', 'red-720'],
        'saturation': [80, 20],
        'luminance': [20, 100]
    },
    name='cubehelix',
    space='hpl',
)
axs[2].format(title='From channel values')
axs[2].pcolormesh(data, cmap=cmap3)

# Display the channels
fig, axs = plot.show_channels(cmap1, cmap2, cmap3, axwidth=1.5, rgb=False)